<a href="https://colab.research.google.com/github/ayush-130904/Stock-Performance-Analyzer/blob/main/stock_performance_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import numpy as np
import pandas as pd
import yfinance as yf

import warnings                        #to ignore the warnings while running the program
warnings.filterwarnings('ignore')

In [144]:
#ask the user for input
ticker_input = input("Enter Stock Name/Symbol").upper().strip()   # upper is used to convert the input into uppercase & strip is used to remove the space/gaps from input
print(f"---Fetching data from {ticker_input}---")

Enter Stock Name/Symbol^NSEI
---Fetching data from ^NSEI---


In [145]:
#download data from yfinance
download = yf.download(ticker_input, period = "1mo", interval = "1d")
stock_name = yf.Ticker(ticker_input).info['longName']

[*********************100%***********************]  1 of 1 completed


In [146]:
prices = download['Close']
df = pd.DataFrame(prices)
df.head(21)

Ticker,^NSEI
Date,
2025-12-08,25960.550781
2025-12-09,25839.650391
2025-12-10,25758.000000
2025-12-11,25898.550781
2025-12-12,26046.949219
2025-12-15,26027.300781
2025-12-16,25860.099609
2025-12-17,25818.550781
2025-12-18,25815.550781


In [147]:
#calculate moving averages
download['MA20'] = df.rolling(window=20).mean()  #20-days moving average

In [148]:
#calculate daily changes in percentage
daily_changes = df.pct_change()
new_daily_changes = daily_changes.dropna()

In [149]:
#calculate volatility - how much a stock jumps around
# step 1- mean of daily chnges
mean_daily_changes = new_daily_changes.mean()

#step 2- calculates the gap for everyday at once
squared_diff = (new_daily_changes - mean_daily_changes)**2

#step 3- Variance
variance = np.mean(squared_diff)

#step 4- volatility
volatility = np.sqrt(variance)

#above steps represents standard deviation

In [150]:
#calculate High and Low Price Range for tmrw
last_price = df.iloc[-1]

#step 1- calc volatility percentage
volatility_pct = np.std(prices.pct_change())*100

#step 2- calc the range
upper_bound = last_price * (1 + volatility_pct)
lower_bound = last_price * (1 - volatility_pct)

In [151]:
#show results
print(f"Stock Name : {stock_name}")
print(f"Current Price : {last_price.item():.2f}rs")
print(f"20 Day Moving Average : {download['MA20'].iloc[-1].item():.2f}rs")
print(f"Daily Risk : {volatility_pct.item():.2f}%")
print(f"Predicted Range for tomorrow: {lower_bound.item():.2f}rs to {upper_bound.item():.2f}rs")

Stock Name : NIFTY 50
Current Price : 26178.70rs
20 Day Moving Average : 26023.98rs
Daily Risk : 0.44%
Predicted Range for tomorrow: 14729.06rs to 37628.34rs


In [152]:
#quick "Advice" logic
if download['Close'].iloc[-1].item() > download['MA20'].iloc[-1].item():
    print("TREND: The stock is currently above its 20-day average (Bullish).")
else:
    print("TREND: The stock is currently below its 20-day average (Bearish).")

TREND: The stock is currently above its 20-day average (Bullish).
